<img width="8%" alt="Geopy.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Geopy.png" style="border-radius: 15%">

# Geopy - Display route itinerary between two locations
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Geopy+-+Display+route+itinerary+between+two+locations:+Error+short+description">Bug report</a>

**Tags:** #geopy #folium #polyline #googlemaps #itinerary #navigation

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Last update:** 2023-07-28 (Created: 2023-07-28)

**Description:** This notebook demonstrates how to display a route initnerary between two locations using `geopy`, `folium`, `polyline` and Google Maps API.

**References:**
- [Nominatim geocoder for OpenStreetMap data](https://geopy.readthedocs.io/en/stable/#nominatim)
- [Folium Mapping: Displaying Markers on a Map](https://towardsdatascience.com/folium-mapping-displaying-markers-on-a-map-6bd56f3e3420)
- [PyPI polyline](https://pypi.org/project/polyline)

## Input

### Import libraries

In [ ]:
from geopy.geocoders import Nominatim
import polyline
import folium
import naas
import requests

### Setup variables
**Mandatory**

Pre-requisite: 
1. Follow the steps in the link - https://developers.google.com/maps/documentation/routes/cloud-setup
2. Sign up for an account with the Routes API provider.
3. Make sure to enable "Routes API" in the "Additional APIs" section in the google cloud console (https://console.cloud.google.com/apis/credentials/key/).

- `api_key`: This variable holds the Google Cloud Platform (GCP) API key. The key is retrieved from the secret variable using the Naas secret manager.
- `origin`: This variable represents the starting location for a journey.
- `destination`: This variable indicates the end location or destination for a journey.


**Optional**

- `zoom_start`: This parameter sets the initial zoom level on the map. A value of 0 indicates the lowest level of zoom.

In [ ]:
# Mandatory
api_key = naas.secret.get("GCP_MAP_API_KEY") or "YOUR_GCP_MAP_API_KEY"  # Read API key from the secret variable GCP_MAP_API_KEY
origin = "Bulgaria Blvd 69, 1404 Manastirski Livadi, Sofia, Bulgaria"
destination = "Sofia, 1680, Bulgaria"

# Optional
zoom_start = 12

## Model

### Get coordinates

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
location1 = geolocator.geocode(origin)
location2 = geolocator.geocode(destination)

### Get points between the two locations

In [ ]:
# Get points using Google Maps API (Routes/Directions)
url = f"https://maps.googleapis.com/maps/api/directions/json?origin={location1.latitude},{location1.longitude}&destination={location2.latitude},{location2.longitude}&key={api_key}"
response = requests.get(url)
data = response.json()
points = data["routes"][0]["overview_polyline"]["points"]

# use polyline to decode points
decoded_points = polyline.decode(points) 

### Create map

In [ ]:
# Create a map centered around the two locations
avg_latitude = (location1.latitude + location2.latitude) / 2
avg_longitude = (location1.longitude + location2.longitude) / 2
map_routes = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=zoom_start)

# Add markers for the specified locations
folium.Marker([location1.latitude, location1.longitude], popup=folium.Popup(location1.address)).add_to(map_routes)
folium.Marker([location2.latitude, location2.longitude], popup=folium.Popup(location2.address)).add_to(map_routes)

# Create route
folium.PolyLine(locations=decoded_points, color='blue').add_to(map_routes)

## Output

### Display map

In [ ]:
map_routes